In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier
from fractions import Fraction
import pickle

In [28]:
# Import data
dfWin = pd.read_csv("./Winners.csv", index_col=0)
dfLose = pd.read_csv("./Losers.csv", index_col=0)
dfWin.head()

,name,desc,cr,immunities,resists,conditions,wins,actions,hp,str,dex,con,int,wis,cha,ac,spd
0,Wraith of Ogre Power,Medium Undead Neutral Evil,5,"Necrotic,Poison,Charmed,Exhaustion,Grappled,Pa...","Acid,Cold,Fire,Lightning,Thunder,Bludgeoning,P...",Gauntlets of Ogre Power,"Werebear, Air Elemental, Troll, Roper, CR 5","Life Drain: +11, 4d8+9 Necrotic & Life Drain",39,19,16,16,12,14,15,13,60
1,Oni,Large Giant Lawful Evil,7,NaN,NaN,"Death Defying,Regeneration",NaN,"Glaive: +6, 10ft, 2d10+4 Slashing\tMultiattack...",55,19,11,16,14,12,15,16,30
2,Giant Ape,Huge Beast Unaligned,7,NaN,NaN,Shielded,NaN,"Fist: +8, 10ft, 3d10+6 Bludgeoning\tRock: +8, ...",69,23,14,18,7,12,7,12,40
3,Young Brass Dragon,Large Dragon Chaotic Good,6,Fire,NaN,Reliable Damage,NaN,"Multiattack: 10ft, Bite, 2 Claw\tBite: +6, 10f...",61,19,10,17,12,11,15,17,80
4,Fire Elemental,Large Elemental Neutral,5,"Fire,Poison,Exhaustion,Grappled,Paralyzed,Petr...","Bludgeoning,Piercing,Slashing","+2 CHA,Heated Body",NaN,"Touch: +5, 2d6+3 Fire, On Fire\tMultiattack: F...",46,10,17,16,6,10,9,13,50


I forgot to put which side the winner and loser were on, therefore I need to randomize where to put the winning creature and where to put the losing creature.
This will allow the model from not constantly predicting the creature on a specific side

In [51]:
import random
# Returns all stat values of a dataframe as a 2d array
def getStatsValues(df: pd.DataFrame):
    return df.loc[:, ["cr", "hp", "str", "dex", "con", "int", "wis", "cha", "ac", "spd"]].values

def convertToFloat(val):
    if type(val) == str:
        return float(Fraction(val))
    else:
        return float(val)

dfWin["cr"] = dfWin.apply(lambda row: convertToFloat(row["cr"]), axis=1)
dfLose["cr"] = dfLose.apply(lambda row: convertToFloat(row["cr"]), axis=1)
# Get Input values
dfWinStats = getStatsValues(dfWin)
dfLoseStats = getStatsValues(dfLose)
# Append input values together
dfStats = np.concatenate([dfWinStats, dfLoseStats], axis=1)
# Generate random list representing which side won and which side lost

# Make the right side outputs equal 1
rightSide = np.ones(dfStats.shape[0]//2)
# Make left side be zero
leftSide = np.zeros(dfStats.shape[0]//2)
# Append all values to targets
targets = np.append(leftSide, rightSide)
# Shuffle results
random.shuffle(targets)


data = []

for i in range(dfWinStats.shape[0]):
    if targets[i]:
        res = np.concatenate([dfLoseStats[i], dfWinStats[i]])
    else:
        res = np.concatenate([dfWinStats[i], dfLoseStats[i]])
    data.append(res)

[0. 0. 1. ... 0. 1. 1.]


In [52]:
# Get the distribution of target values

one = 0
zero = 0

for val in targets:
    if val:
        one += 1
    else:
        zero += 1

print(f"One: {one}")
print(f"Zero: {zero}")

One: 1018
Zero: 1018


In [53]:
splitSize = 0.8

X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=(1-splitSize), random_state=47)


In [54]:
print(len(X_test[0]))

20


In [55]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

# params = { 'hidden_layer_sizes': [(200, 100, 90), (2000, 500, 80), (2000, 1500, 80), (3000, 1000, 80)]
#             'solver': ['lbfgs']
# }

clf = MLPClassifier(hidden_layer_sizes=(2000, 1000, 80), max_iter=100000, solver='lbfgs', learning_rate_init=1e-4, momentum=0.9)

#gs = GridSearchCV(clf)

clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(2000, 1000, 80), learning_rate_init=1e-05,
              max_iter=100000, solver='lbfgs')

In [56]:
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))



              precision    recall  f1-score   support

         0.0       0.57      0.63      0.60       198
         1.0       0.61      0.56      0.58       210

    accuracy                           0.59       408
   macro avg       0.59      0.59      0.59       408
weighted avg       0.59      0.59      0.59       408



In [57]:


# save the classifier to a file
with open("classifier_lbfgs64_perc.pkl", "wb") as file:
    pickle.dump(clf, file)

In [58]:
# load the saved classifier from the file
with open("classifier_lbfgs64_perc.pkl", "rb") as file:
    clf = pickle.load(file)

predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.57      0.63      0.60       198
         1.0       0.61      0.56      0.58       210

    accuracy                           0.59       408
   macro avg       0.59      0.59      0.59       408
weighted avg       0.59      0.59      0.59       408



In [400]:
clf

MLPClassifier(hidden_layer_sizes=(2000, 1000, 80), learning_rate_init=1e-05,
              max_iter=10000000)